<h2></h2>
<p>In this notebook I calculate all the posterior distributions for all the goalies playing at that time and match those similar to the selected goalie</p>

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime as dt, timedelta
from dateutil.relativedelta import *
import re
import sqlite3
import scipy.stats as stats
pd.set_option('display.max_rows', 500)

<h3>SQLite Handling functions</h3>
<p>Use Pandas to import dataframe into SQLlite</p>

In [5]:
DB = "..\\Data\\hockey_data_goalies.db"

#SQL Handle functions
def run_query(q):
    with sqlite3.connect(DB) as conn:
        x = pd.read_sql(q,conn)
    return x
   
#send command
def run_command(c):
    with sqlite3.connect(DB) as conn:
        conn.isolation_level = None
        return conn.execute(c)
    
#show tables
def show_tables():
    r = """
    Select name,type
    FROM sqlite_master
    WHERE type in ("table","view");
        """
    return run_query(r)

In [18]:
a = 231.14
b = 24.2
q = """SELECT player_id,SUM(saves) saves, SUM(goals_against) goals
        FROM player_log log
        LEFT JOIN player_list pl ON pl.unique_id= log.player_id
        WHERE (CAST(SUBSTR(date_game,1,4) AS FLOAT)+CAST(SUBSTR(date_game,6,7) AS FLOAT)/12) > {0}
        AND date_game < \"{1}\"
        GROUP BY player_id
        """.format(2016.66,'2017-01-17')
active_players = run_query(q)
active_players['alpha_prime'] = active_players['saves'] + a
active_players['beta_prime'] = active_players['goals'] + b
active_players['mean'] = active_players['alpha_prime']/(active_players['alpha_prime'] +active_players['beta_prime'])
active_players['var'] = np.sqrt((active_players['alpha_prime']*active_players['beta_prime'])/((active_players['alpha_prime'] +active_players['beta_prime'])**2+(1+active_players['alpha_prime'] +active_players['beta_prime'])))

In [19]:
active_players

,player_id,saves,goals,alpha_prime,beta_prime,mean,var
0,allenja01,757,84,988.14,108.2,0.901308,0.298112
1,andercr01,568,47,799.14,71.2,0.918193,0.273913
2,anderfr01,1020,90,1251.14,114.2,0.916358,0.276749
3,bernijo01,339,36,570.14,60.2,0.904496,0.293677
4,berubje02,75,8,306.14,32.2,0.904829,0.293016
5,bishobe01,610,60,841.14,84.2,0.909006,0.287445
6,bobrose01,894,66,1125.14,90.2,0.925782,0.262017
7,budajpe01,833,75,1064.14,99.2,0.914728,0.279166
8,campbja01,5,0,236.14,24.2,0.907045,0.289812
9,condomi01,517,46,748.14,70.2,0.914217,0.279873


In [24]:
#pick a random goalie
player = active_players.iloc[np.random.randint(0,active_players.shape[0])]

In [32]:
#difference
active_players['distance'] = np.sqrt(((active_players['mean']-player['mean'])/player['mean'])**2+((active_players['var']-player['var'])/player['var'])**2)
#get top three recommendations
active_players.sort_values('distance')[1:4]

,player_id,saves,goals,alpha_prime,beta_prime,mean,var,distance
72,wardca01,920,86,1151.14,110.2,0.912633,0.282261,0.000436
61,reimeja01,466,43,697.14,67.2,0.912081,0.282992,0.003094
48,millery01,724,68,955.14,92.2,0.911967,0.283207,0.003866
